# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [27]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [28]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [29]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])



8056
['Stephen Lynch', 'Logged In', 'Jayden', 'M', '0', 'Bell', '182.85669', 'free', 'Dallas-Fort Worth-Arlington, TX', 'PUT', 'NextSong', '1.54099E+12', '829', "Jim Henson's Dead", '200', '1.54354E+12', '91']


In [30]:
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))



In [31]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [32]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect() #To establish connection and begin executing queries, need a session
except Exception as e:
    print(e)



#### Create Keyspace

In [33]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify7 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [34]:
try:
    session.set_keyspace('sparkify7')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [36]:
def values_query(quantity,condition):
    """
    - Creates a string input values for the insert query
    - Params:
        - Quantity: amount of parameters (%s) to include in the string
    - Returns a string in the form of 'VALUES (%s,%s,...,%s)'
    """
    if condition:
        query = 'VALUES ('
    else:
        query = '('
    i = 0
    for i in range(0,quantity):
        if i+1==quantity:
            query+='%s'
        else:
            query+='%s,'
        i+=1
    query+=(')')
    return query

def create_tables(session,table_name,columns):
    query = "drop table IF EXISTS " + table_name
    try:
        rows = session.execute(query)
        print('Table dropped')
    except Exception as e:
        print(e)

    query = "CREATE TABLE IF NOT EXISTS " + table_name
    query = query + columns
    try:
        session.execute(query)
        print('Table created')
    except Exception as e:
        print(e)

def insert_data_tables(session, file, table_name, columns, amount_values, values):
    print('Inserting data for table',table_name)
    counter = 0
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            if counter % 1000 ==0:
                print(counter)
            query = "INSERT INTO " + table_name + " " + columns
            query = query + values_query(amount_values,True)
            values_full = []
            for k,v in values.items():
                if v == 'int':
                    values_full.append(int(line[k]))
                else:
                    values_full.append(str(line[k]))
            try:
                session.execute(query, values_full)
            except Exception as e:
                print(e)
            counter += 1
    print('Done inserting',counter,'files')


In [37]:
creates_dict={
    'music_app_history1':'(sessionId int, itemInSession int, artist text, song text, length text, PRIMARY KEY (sessionId, itemInSession))',
    'music_app_history2':'(userId int, sessionId int, itemInSession int, name_artist text, song text, user_first_name text, user_last_name text, PRIMARY KEY (userId, sessionId, itemInSession))',
    'music_app_history3':'(song text, user_id int, user_first_name text, user_last_name text, PRIMARY KEY (song,user_id))'
}

for key, value in creates_dict.items():
    create_tables(session,key,value)
    
insert_data_tables_dict={
    'music_app_history1':['(sessionId,itemInSession,artist,song,length)',5,
                          {8:'int',3:'int',0:'text',9:'text',5:'text'}],
    'music_app_history2':['(userId, sessionId, itemInSession, name_artist, song, user_first_name, user_last_name)',7,
                          {10:'int',8:'int',3:'int',0:'text',9:'text',1:'text',4:'text'}],
    'music_app_history3':['(song, user_id, user_first_name, user_last_name)',4,
                          {9:'text',10:'int',1:'text',4:'text'}]
}

file = 'event_datafile_new.csv'
for key, value in insert_data_tables_dict.items():
    insert_data_tables(session,file,key,value[0],value[1],value[2])

Table dropped
Table created
Table dropped
Table created
Table dropped
Table created
Insert query for table music_app_history1
0
1000
2000
3000
4000
5000
6000
Done inserting 6820 files
Insert query for table music_app_history2
0
1000
2000
3000
4000
5000
6000
Done inserting 6820 files
Insert query for table music_app_history3
0
1000
2000
3000
4000
5000
6000
Done inserting 6820 files


#### Do a SELECT to verify that the data have been inserted into each table

In [38]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "select artist, song, length from music_app_history1 WHERE sessionId=338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [39]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "select itemInSession, name_artist, song, user_first_name, user_last_name from music_app_history2 WHERE userId=10 and sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

print("Verifying query and itemInSession sort \n")
for row in rows:
    print (row.iteminsession, row.name_artist, row.song, row.user_first_name, row.user_last_name)

Verifying query and itemInSession sort 

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [40]:
##Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "select user_first_name, user_last_name from music_app_history3 WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    
    print (row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [44]:
for i in range (1,4):
    query = ("drop table music_app_history" + str(i))
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [45]:
session.shutdown()
cluster.shutdown()